In [155]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [114]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=False)

In [374]:
print([np.sum(tX[:,i]==-999) for i in range(tX.shape[1])])

[38114, 0, 0, 0, 177457, 177457, 177457, 0, 0, 0, 0, 0, 177457, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99913, 99913, 99913, 177457, 177457, 177457, 0]


In [175]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

In [545]:
#print(tX.shape)
#print(y.shape)

## Do your thing crazy machine learning thing here :) ...

In [143]:
ratio = 0.8
seed = 6
tx_training, tx_testing, y_training, y_testing = split_data(tX, y, ratio, seed)

In [144]:
tx_training = np.delete(tx_training, (4,5,6,12,26,27,28), axis=1)
tx_training, col_median_tX = correction_missing_values(tx_training)
#print([(np.max(tX[:,i]),np.min(tX[:,i]),np.mean(tX[:,i]),np.median(tX[:,i])) for i in range(tX.shape[1])])
#plt.boxplot(tx_training)

In [145]:
tx_training, perc_25, perc_75, col_median_tX = remove_outliers(tx_training)

In [28]:
#plt.boxplot(tX)

In [146]:
tx_training, max_, min_ = normalize(tx_training)
tx_training, mean, std = standardize(tx_training)

In [147]:
tx_testing = np.delete(tx_testing, (4,5,6,12,26,27,28), axis=1)
tx_testing = correction_missing_values_test(tx_testing, col_median_tX)
tx_testing = remove_outliers_test(tx_testing, perc_25, perc_75, col_median_tX)
tx_testing = (tx_testing - min_) / (max_ - min_)
tx_testing = (tx_testing - mean) / std

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

clf_logistic = LogisticRegression(random_state=0).fit(tx_training, y_training)
clf_SVC = SVC(gamma='auto').fit(tx_training, y_training)
print("Accuracy from scikit learn - logistic regression: "+str(round(100*clf_logistic.score(tx_training, y_training),5))+' %\n')
print("Accuracy from scikit learn - Support Vector Machine: "+str(round(100*clf_SVC.score(tx_training, y_training),5))+' %\n')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy from scikit learn - logistic regression: 81.8 %

Accuracy from scikit learn - Support Vector Machine: 93.6 %



In [62]:
from logistic_regression import *
#Using our code
#Logistic regression

gamma = 0.0001
n_iter = 90

tx_training = np.c_[np.ones((y_training.shape[0], 1)), tx_training]
tx_testing = np.c_[np.ones((y_testing.shape[0], 1)), tx_testing]

w = np.random.rand(tx_training.shape[1])
for i in range(n_iter):
    w, loss = learning_by_gradient_descent(y_training, tx_training, w, gamma)
    if i % 10 == 0:
        print("Current iteration={iter}, training loss={l_tr}, testing loss={l_te}".format(iter=i, l_tr=loss, 
                                                                                                         l_te=calculate_loss(y_testing, tx_testing, w)))
print(w, loss)
print("training loss={l_tr}".format(l_tr=calculate_loss(y_training, tx_training, w)))
print("testing loss={l_te}".format(l_te=calculate_loss(y_testing, tx_testing, w)))

print("Accuracy from our code:\n")
print("Logistic regression: "+str(round(100*np.sum(predict_labels(w, tx_testing, True)==y_testing)/len(y_testing),5))+" %\n")

Current iteration=0, training loss=5521.413283753469, testing loss=1260.454865442583
Current iteration=10, training loss=2831.4255880359387, testing loss=687.4562267101987
Current iteration=20, training loss=2204.258403944562, testing loss=563.861775675578
Current iteration=30, training loss=2019.4559514421726, testing loss=529.3800951470937
Current iteration=40, training loss=1947.8456766483139, testing loss=517.0578460250651
Current iteration=50, training loss=1915.3428246331996, testing loss=512.0994962269394
Current iteration=60, training loss=1898.8732012918622, testing loss=510.01874568679307
Current iteration=70, training loss=1889.8459935832577, testing loss=509.1796589371522
Current iteration=80, training loss=1884.6050025078605, testing loss=508.9050868244675
[-0.88978064  1.02862523 -0.70042264 -0.10334482 -0.18762193  0.05445285
 -0.09676001 -0.0338816  -0.30386946  0.36810643  0.17987915 -0.01235336
  0.03359423  0.04786577 -0.07723513  0.08833339 -0.06242726  0.01086544
 

In [63]:
y_pred = predict_labels(w, tx_testing, True)
#print(y_pred)

In [64]:
print(len(y_pred))
print(np.sum([y_pred==1]))
print(np.sum([y_testing==1]))

1000
283
338


In [65]:
#test on all data points from train.csv
y_, tX_, ids_ = load_csv_data('../data/train.csv', sub_sample=False)
print(y_.shape)

(250000,)


In [66]:
tX_ = np.delete(tX_, (4,5,6,12,26,27,28), axis=1)
tX_ = correction_missing_values_test(tX_, col_median_tX)
tX_ = remove_outliers_test(tX_, perc_25, perc_75, col_median_tX)
tX_ = (tX_ - min_) / (max_ - min_)
tX_ = (tX_ - mean) / std
tX_ = np.c_[np.ones((tX_.shape[0], 1)), tX_]

In [67]:
y_pred = predict_labels(w, tX_, True)
print(np.sum([y_pred==1]))
print(np.sum([y_==1]))
print("Logistic regression: "+str(round(100*np.sum(y_pred==y_)/len(y_),5))+" %\n")

71465
85667
Logistic regression: 77.5576 %



In [100]:
from reg_logistic_regression import *
#Reguralized logistic regression

gamma = 0.0001
lamda_ = 0.105
n_iter = 90

tx_training = np.c_[np.ones((y_training.shape[0], 1)), tx_training]
tx_testing = np.c_[np.ones((y_testing.shape[0], 1)), tx_testing]

w_reg = np.random.rand(tx_training.shape[1])
for i in range(n_iter):
    w_reg, loss = learning_by_penalized_gradient(y_training, tx_training, w_reg, gamma, lamda_)
    if i % 10 == 0:
        print("Current iteration={iter}, training loss={l_tr}, testing loss={l_te}".format(iter=i, l_tr=loss, 
                                                                                                         l_te=calculate_loss(y_testing, tx_testing, w_reg)))
print(w_reg, loss)
print("training loss={l_tr}".format(l_tr=calculate_loss(y_training, tx_training, w_reg)))
print("testing loss={l_te}".format(l_te=calculate_loss(y_testing, tx_testing, w_reg)))

print("After regularization: "+str(round(100*np.sum(predict_labels(w_reg, tx_testing, True)==y_testing)/len(y_testing),5))+" %\n")

Current iteration=0, training loss=5522.318831095153, testing loss=1260.4333781867192
Current iteration=10, training loss=2831.5040238577826, testing loss=687.3783248511893
Current iteration=20, training loss=2204.334581461228, testing loss=563.8113249033254
Current iteration=30, training loss=2019.5940856386665, testing loss=529.3483253001523
Current iteration=40, training loss=1948.0357998134643, testing loss=517.0361769260752
Current iteration=50, training loss=1915.5701962888809, testing loss=512.0828977498253
Current iteration=60, training loss=1899.1268346255213, testing loss=510.0044088618023
Current iteration=70, training loss=1890.118269395124, testing loss=509.16597625588525
Current iteration=80, training loss=1884.8906396331927, testing loss=508.8911341349611
[-0.8892887   1.02793143 -0.70016142 -0.10307377 -0.18741807  0.05434073
 -0.0966743  -0.03384179 -0.30361892  0.36787893  0.17993639 -0.01237023
  0.03351597  0.04774421 -0.07715804  0.08822358 -0.06233701  0.01085407


In [101]:
y_pred = predict_labels(w_reg, tx_testing, True)
#print(y_pred)

In [102]:
print(len(y_pred))
print(np.sum([y_pred==1]))
print(np.sum([y_testing==1]))

1000
283
338


In [103]:
#test on all data points from train.csv
y_, tX_, ids_ = load_csv_data('../data/train.csv', sub_sample=False)

In [104]:
tX_ = np.delete(tX_, (4,5,6,12,26,27,28), axis=1)
tX_ = correction_missing_values_test(tX_, col_median_tX)
tX_ = remove_outliers_test(tX_, perc_25, perc_75, col_median_tX)
tX_ = (tX_ - min_) / (max_ - min_)
tX_ = (tX_ - mean) / std
tX_ = np.c_[np.ones((tX_.shape[0], 1)), tX_]

In [105]:
y_pred = predict_labels(w_reg, tX_, True)
print(np.sum([y_pred==1]))
print(np.sum([y_==1]))
print("Regularized logistic regression: "+str(round(100*np.sum(y_pred==y_)/len(y_),5))+" %\n")

71468
85667
Regularized logistic regression: 77.554 %



In [24]:
from grid_search import *
# optimising the lambda for regularized logistic regression 

gamma = 0.0001
n_iter = 10000
ratio = 0.8
seed = 6
n_intervals = 20

tX = np.c_[np.ones((y.shape[0], 1)), tX]

tx_training, tx_testing, y_training, y_testing = split_data(tX, y, ratio, seed)

w_reg = np.random.rand(tX.shape[1])

lambdas = generate_lambda(n_intervals)
min_loss, best_lambda = grid_search(y_training, y_testing, tx_training, tx_testing, w_reg, lambdas, gamma, n_iter)

In [ ]:
print("The optimum lambda is : " + str(best_lambda))
print("The corresponding loss is : " + str(min_loss))

In [148]:
from least_squares import *
#least squares

#try polynomial
degree = 3
tx_training = build_poly(tx_training, degree)
tx_training = np.c_[np.ones((y_training.shape[0], 1)), tx_training]
tx_testing = build_poly(tx_testing, degree)
tx_testing = np.c_[np.ones((y_testing.shape[0], 1)), tx_testing]

w_lsq, loss = least_squares(y_training, tx_training)
print("weights={w}, loss={l}".format(w=w_lsq, l=loss))
print("training loss={l_tr}".format(l_tr=compute_loss(y_training, tx_training, w_lsq)))
print("testing loss={l_te}".format(l_te=compute_loss(y_testing, tx_testing, w_lsq)))
print("After least squares: "+str(round(100*np.sum(predict_labels(w_lsq, tx_testing, False)==y_testing)/len(y_testing),5))+" %\n")

weights=[ 3.66598441e-01  2.09398782e-01 -3.05068879e-03 -1.99818081e-02
 -1.83384785e-01 -2.66200489e-02  3.07285822e-02  4.52252769e-02
 -4.47537037e-02 -3.10521859e-03 -5.95299805e-02 -3.76918551e-03
  7.79476558e-03 -2.41872499e-02 -1.72349806e-02 -2.32072324e-04
 -1.85236296e-02  2.80616785e-02 -4.32533197e-03  5.53024378e-03
  4.05712481e-03 -3.72715217e-03 -2.29538811e-02  9.83291248e-03
 -5.47150075e-04  5.86382336e-02  3.54795335e-02 -1.12674819e-02
  5.87072500e-02 -3.34374623e-03 -2.06295721e-03 -4.72162526e-03
 -1.72343279e-02 -4.89215164e-04  7.56382855e-03 -8.47417844e-03
 -2.58754396e-03 -2.59801853e-02  2.87535229e-03  4.10879532e-04
  2.79087785e-03 -3.04015839e-02 -4.17405857e-03  1.79978944e-02
 -6.36495793e-03 -2.52412325e-03  3.76673750e-03  8.94330230e-03
 -1.46389377e-03  3.42666271e-02  1.50008577e-03 -1.81931039e-02
  3.50202375e-04 -6.25274602e-03  9.15348524e-04  7.63380754e-02
  2.34957727e-02 -3.80639468e-02  2.57594706e-02 -1.55082641e-02
  3.98318897e-03 

In [149]:
y_pred = predict_labels(w_lsq, tx_testing, False)
#print(y_pred)

In [150]:
print(len(y_pred))
print(np.sum([y_pred==1]))
print(np.sum([y_testing==1]))

1000
287
338


In [151]:
#test on all data points from train.csv
y_, tX_, ids_ = load_csv_data('../data/train.csv', sub_sample=False)
print(y_.shape)

(250000,)


In [152]:
tX_ = np.delete(tX_, (4,5,6,12,26,27,28), axis=1)
tX_ = correction_missing_values_test(tX_, col_median_tX)
tX_ = remove_outliers_test(tX_, perc_25, perc_75, col_median_tX)
tX_ = (tX_ - min_) / (max_ - min_)
tX_ = (tX_ - mean) / std
tX_ = build_poly(tX_, degree)
tX_ = np.c_[np.ones((tX_.shape[0], 1)), tX_]

In [153]:
y_pred = predict_labels(w_lsq, tX_, False)
print(np.sum([y_pred==1]))
print(np.sum([y_==1]))
print("Least squares: "+str(round(100*np.sum(y_pred==y_)/len(y_),5))+" %\n")

75823
85667
Least squares: 80.3456 %



In [129]:
#choosing polynomial degree
def polynomial_regression():
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    # define parameters
    degrees = [1, 2, 3, 4, 5, 6, 7, 8]

    for ind, degree in enumerate(degrees):
        tx_training_ = build_poly(tx_training, degree)
        tx_training_ = np.c_[np.ones((y_training.shape[0], 1)), tx_training_]
        tx_testing_ = build_poly(tx_testing, degree)
        tx_testing_ = np.c_[np.ones((y_testing.shape[0], 1)), tx_testing_]

        weights, mse = least_squares(y_training, tx_training_)
        print("training loss={l_tr}".format(l_tr=compute_loss(y_training, tx_training_, weights)))
        print("testing loss={l_te}".format(l_te=compute_loss(y_testing, tx_testing_, weights)))
        rmse = np.sqrt(2 * mse)

        print("Processing {i}th experiment, degree={d}, training rmse={tr_loss}, testing rmse={te_loss}".format(
              i=ind + 1, d=degree, tr_loss=rmse, te_loss=compute_loss(y_testing, tx_testing_, weights)))
        
polynomial_regression()

training loss=0.07906238228549702
testing loss=0.08644314556672941
Processing 1th experiment, degree=1, training rmse=0.3976490469886656, testing rmse=0.08644314556672941
training loss=0.07257482112180884
testing loss=0.08039932152981177
Processing 2th experiment, degree=2, training rmse=0.380985094516331, testing rmse=0.08039932152981177
training loss=0.0697153219398077
testing loss=0.07828847199031279
Processing 3th experiment, degree=3, training rmse=0.3734041294356764, testing rmse=0.07828847199031279
training loss=0.06881541109575426
testing loss=0.07782891476206491
Processing 4th experiment, degree=4, training rmse=0.3709862830233869, testing rmse=0.07782891476206491
training loss=30.954725931273213
testing loss=34.610948733122264
Processing 5th experiment, degree=5, training rmse=7.8682559606653895, testing rmse=34.610948733122264
training loss=5.199442406434914
testing loss=5.231264891349185
Processing 6th experiment, degree=6, training rmse=3.2247301922594747, testing rmse=5.2

In [173]:
def cross_validation(y, x, k_indices, k, degree):
    # get k'th subgroup in test, others in train: TODO
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]

    tr_indice = tr_indice.reshape(-1)

    y_testing = y[te_indice]
    y_training = y[tr_indice]
    tx_testing = tX[te_indice]
    tx_training = tX[tr_indice]

    tx_training = np.delete(tx_training, (4,5,6,12,26,27,28), axis=1)
    tx_training, col_median_tX = correction_missing_values(tx_training)
    tx_training, perc_25, perc_75, col_median_tX = remove_outliers(tx_training)
    tx_training, max_, min_ = normalize(tx_training)
    tx_training, mean, std = standardize(tx_training)
    
    tx_testing = np.delete(tx_testing, (4,5,6,12,26,27,28), axis=1)
    tx_testing = correction_missing_values_test(tx_testing, col_median_tX)
    tx_testing = remove_outliers_test(tx_testing, perc_25, perc_75, col_median_tX)
    tx_testing = (tx_testing - min_) / (max_ - min_)
    tx_testing = (tx_testing - mean) / std
    
    tx_training_ = build_poly(tx_training, degree)
    tx_training_ = np.c_[np.ones((y_training.shape[0], 1)), tx_training_]
    tx_testing_ = build_poly(tx_testing, degree)
    tx_testing_ = np.c_[np.ones((y_testing.shape[0], 1)), tx_testing_]

    w_lsq, loss = least_squares(y_training, tx_training_)

    loss_tr = np.sqrt(2 * compute_loss(y_training, tx_training_, w_lsq))
    loss_te = np.sqrt(2 * compute_loss(y_testing, tx_testing_, w_lsq))

    return loss_tr, loss_te, w

In [174]:
def best_degree_selection(degrees, k_fold, seed = 1):
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    
    #for each degree, we compute the best lambdas and the associated rmse
    #best_lambdas = []
    best_rmses = []
    #vary degree
    for degree in degrees:
        # cross validation
        #rmse_te = []
        #for lambda_ in lambdas:
        rmse_te_tmp = []
        for k in range(k_fold):
            _, loss_te, _ = cross_validation(y, tX, k_indices, k, degree)
            rmse_te_tmp.append(loss_te)
        best_rmses.append(np.mean(rmse_te_tmp))
        
        #ind_lambda_opt = np.argmin(rmse_te)
        #best_lambdas.append(lambdas[ind_lambda_opt])
        #best_rmses.append(rmse_te[ind_lambda_opt])
        
    ind_best_degree =  np.argmin(best_rmses)
        
    return degrees[ind_best_degree]

best_degree_selection(np.arange(1,11), 4)

3

In [23]:
from ridge_regression import *
#ridge regression


degree = 3
tx_training = build_poly(tx_training, degree)
tx_training = np.c_[np.ones((y_training.shape[0], 1)), tx_training]
tx_testing = build_poly(tx_testing, degree)
tx_testing = np.c_[np.ones((y_testing.shape[0], 1)), tx_testing]

lambdas = np.logspace(-5, 0, 15)
for ind, lambda_ in enumerate(lambdas):
    # ridge regression with a given lambda
    weight_ridge, loss = ridge_regression(y_training, tx_training, lambda_)
    print("lambda={lam:.5f}, training loss={l_tr:.5f}, testing loss={l_te:.5f}".format(lam=lambda_, l_tr=loss, 
                                                                                       l_te=compute_loss(y_testing, tx_testing, weight_ridge)))
    
print("weights={w}, loss={l}".format(w=weight_ridge, l=loss))
print("training loss={l_tr}".format(l_tr=compute_loss(y_training, tx_training, weight_ridge)))
print("testing loss={l_te}".format(l_te=compute_loss(y_testing, tx_testing, weight_ridge)))

print("After ridge regression: "+str(round(100*np.sum(predict_labels(weight_ridge, tx_testing, False)==y_testing)/len(y_testing),5))+" %\n")

lambda=0.00001, training loss=0.06978, testing loss=0.07825
lambda=0.00002, training loss=0.06978, testing loss=0.07825
lambda=0.00005, training loss=0.06979, testing loss=0.07825
lambda=0.00012, training loss=0.06980, testing loss=0.07825
lambda=0.00027, training loss=0.06984, testing loss=0.07826
lambda=0.00061, training loss=0.06992, testing loss=0.07826
lambda=0.00139, training loss=0.07009, testing loss=0.07828
lambda=0.00316, training loss=0.07044, testing loss=0.07832
lambda=0.00720, training loss=0.07110, testing loss=0.07844
lambda=0.01638, training loss=0.07226, testing loss=0.07867
lambda=0.03728, training loss=0.07425, testing loss=0.07917
lambda=0.08483, training loss=0.07748, testing loss=0.08023
lambda=0.19307, training loss=0.08242, testing loss=0.08234
lambda=0.43940, training loss=0.08946, testing loss=0.08616
lambda=1.00000, training loss=0.09867, testing loss=0.09211
weights=[ 2.37134529e-02  3.66651444e-02  2.33090657e-02  1.26343821e-02
 -3.17986506e-02  2.4213742

In [200]:
from gradient_descent import *

ratio = 0.8
seed = 6

tX = np.c_[np.ones((y.shape[0], 1)), tX]

tx_training, tx_testing, y_training, y_testing = split_data(tX, y, ratio, seed)

# Define the parameters of the algorithm.
max_iters = 5000
gamma = 0.1

# Initialization
w_initial = np.random.rand(tX.shape[1])

# Start gradient descent.
gradient_ws, gradient_losses= gradient_descent(y_training, tx_training, w_initial, max_iters, gamma)
print("weights={w}, training loss={l}".format(w=gradient_ws, l=gradient_losses))
print("training loss={l_tr}".format(l_tr=compute_loss(y_training, tx_training, gradient_ws)))
print("testing loss={l_te}".format(l_te=compute_loss(y_testing, tx_testing, gradient_ws)))
print("After gradient descent: "+str(round(100*np.sum(predict_labels(gradient_ws, tx_testing, False)==y_testing)/len(y_testing),5))+" %\n")

Current iteration=0, loss=12.291883010453807
Current iteration=1000, loss=0.10071737801961865
Current iteration=2000, loss=0.09508135294572446
Current iteration=3000, loss=0.09235130154805092
Current iteration=4000, loss=0.09083357303280999
weights=[-0.63533999 -0.1550554   0.25046177 -1.56523086 -0.19626246  0.80848127
  0.5743233   0.39290491 -0.62852223  0.93930804 -0.39008463  0.67514876
 -0.56810384  1.03366328  0.58848327  1.40398672 -0.10265195  0.26138214
 -0.56507097  0.06959221  0.31976244 -0.09861862  0.2379758   0.12575469
  0.39449208  0.2449233   0.15825706  0.3090895  -0.36485586 -0.01063269
  0.13791344 -0.24721618], training loss=0.08989135678696848
training loss=0.08989059311055095
testing loss=0.0916365284054964
After gradient descent: 72.1 %



In [30]:
from stochastic_gradient_descent import *

ratio = 0.8
seed = 6

tX = np.c_[np.ones((y.shape[0], 1)), tX]

tx_training, tx_testing, y_training, y_testing = split_data(tX, y, ratio, seed)

# Define the parameters of the algorithm.
max_iters = 5000
gamma = 0.1

# Initialization
w_initial = np.random.rand(tX.shape[1])

# Start SGD.
sgd_ws, sgd_losses = stochastic_gradient_descent(y_training, tx_training, w_initial, max_iters, gamma)
print("weights={w}, training loss={l}".format(w=sgd_ws, l=sgd_losses))
print("training loss={l_tr}".format(l_tr=compute_loss(y_training, tx_training, sgd_ws)))
print("testing loss={l_te}".format(l_te=compute_loss(y_testing, tx_testing, sgd_ws)))
print("After stochastic gradient descent: "+str(round(100*np.sum(predict_labels(sgd_ws, tx_testing, False)==y_testing)/len(y_testing),5))+" %\n")

Current iteration=0, loss=10.684155249537138
Current iteration=100, loss=0.11611652153666797
Current iteration=200, loss=0.14554730629189477
Current iteration=300, loss=0.15729584012939785
Current iteration=400, loss=0.14270265710658306
Current iteration=500, loss=0.14298708683238764
Current iteration=600, loss=0.10556115934044442
Current iteration=700, loss=0.10730606578847834
Current iteration=800, loss=0.14551074619988377
Current iteration=900, loss=0.21453318010213285
Current iteration=1000, loss=0.11551455058310207
Current iteration=1100, loss=0.16175518591596363
Current iteration=1200, loss=0.1205090926539504
Current iteration=1300, loss=0.12327826509625146
Current iteration=1400, loss=0.10193669583300485
Current iteration=1500, loss=0.0981311189563209
Current iteration=1600, loss=0.10670949634004609
Current iteration=1700, loss=0.10871143412603702
Current iteration=1800, loss=0.09593688576563207
Current iteration=1900, loss=0.09624031797805332
Current iteration=2000, loss=0.1446

## Generate predictions and save ouput in csv format for submission:

In [70]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test = np.delete(tX_test, (4,5,6,12,26,27,28), axis=1)
tX_test = correction_missing_values_test(tX_test, col_median_tX)
tX_test = remove_outliers_test(tX_test, perc_25, perc_75, col_median_tX)
tX_test = (tX_test - min_) / (max_ - min_)
tX_test = (tX_test - mean) / std
#tX_test = build_poly(tX_test, 3)
tX_test = np.c_[np.ones((tX_test.shape[0], 1)), tX_test]

In [71]:
#OUTPUT_PATH = 'test_lsq4.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w, tX_test, True)
y_pred[np.where(y_pred == 0)] = -1
#create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [72]:
print(np.sum([y_pred==1]))
print(np.sum([y_pred==-1]))

162573
405665
